In [1]:
import os # 파이썬을 이용해 파일을 복사하거나 디렉터리를 생성하고 특정 디렉터리 내의 파일 목록을 구하고자 할 때 사용
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision # torchvision package : 컴퓨터 비전을 위한 유명 데이터셋, 모델 아키텍처, 이미지 변형등을 포함
import torch.nn as nn # nn : neural netwroks (define class) attribute를 활용해 state를 저장하고 활용
import torch.optim as optim # 최적화 알고리즘
import torch.nn.functional as F # (define function) 인스턴스화 시킬 필요없이 사용 가능
from PIL import Image
from torchvision import transforms, datasets # transforms : 데이터를 조작하고 학습에 적합하게 만듦.
from torch.utils.data import Dataset, DataLoader
# dataset : 샘플과 정답(label)을 저장
# DataLoader : Dataset 을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)로 감싼다.

In [2]:
epochs = 10 # 훈련 반복수
batch_size = 512 # 배치 크기

device = ("cuda" if torch.cuda.is_available() else "cpu") # device 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] # 총 10개의 클래스

print(torch.__version__)
print(device)

2.0.1+cu118
cuda


In [3]:
transform = transforms.Compose([
    transforms.Resize(227), # Compose : transforms 리스트 구성
    # 227x227 : input image(in alexnet) but fashionMNIST's input image : 28x28
    transforms.ToTensor()]) # ToTensor : PIL image or numpy.ndarray를 tensor로 바꿈 # 딥러닝 모델에 데이터 입력하기 위해서는 텐서로 변환해야 함

training_data = datasets.FashionMNIST(
    root="data", # data가 저장될 경로(path)
    train=True, # training dataset
    download=True, # 인터넷으로부터 데이터 다운
    transform=transform # feature 및 label 변환(transformation) 지정
)

validation_data = datasets.FashionMNIST(
    root="data",
    train=False, # test dataset
    download=True,
    transform=transform
)

100%|██████████| 26421880/26421880 [00:02<00:00, 11824493.21it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 210231.25it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3760385.73it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21086208.00it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
# (class) DataLoader(dataset, batch_size, shuffle, ...)
# DataLoader는 주어진 데이터셋을 batch로 나누고 데이터를 미니배치로 제공
training_loader = DataLoader(training_data, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=64, shuffle=True)

In [6]:
# DataLoader에서 첫 번째 미니배치 가져오기
first_batch = next(iter(training_loader))

# 미니배치의 데이터와 레이블 가져오기
images, labels = first_batch

# 미니배치의 크기 확인
print(images.shape)  # 이미지 데이터의 크기 출력
print(labels.shape)  # 레이블 데이터의 크기 출력

torch.Size([64, 1, 227, 227])
torch.Size([64])


iter(training_loader)를 호출하여 training_loader 데이터 로더를 반복 가능한(iterable) 객체로 변환합니다. 이를 통해 데이터 로더의 각 미니배치에 순차적으로 액세스할 수 있는 반복자(iterator)를 생성합니다.

next(iter(training_loader))를 호출하여 반복자에서 다음 미니배치를 가져옵니다. 이 작업은 데이터 로더에서 다음 미니배치를 로드하는 역할을 합니다.

first_batch 변수에 이 미니배치를 저장합니다. 이 미니배치는 텐서로 구성된 이미지 데이터와 해당 이미지의 레이블 데이터를 포함하고 있습니다.

즉, 위의 코드는 training_loader로부터 첫 번째 미니배치를 가져와서 first_batch 변수에 저장하는 것입니다. 이렇게 하면 첫 번째 미니배치의 데이터를 사용하여 모델을 학습하거나 분석할 수 있습니다. 첫 번째 미니배치 이후에는 다음 미니배치를 가져오기 위해 다시 next(iter(training_loader))를 호출할 수 있습니다.

In [7]:
class fashion_mnist_alexnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(

            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0),
            # 4D tensor : [number_of_kernels, input_channels, kernel_width, kernel_height]
            # = 96x1x11x11
            # input size : 1x227x227
            # input size 정의 : (N, C, H, W) or (C, H, W)
            # W' = (W-F+2P)/S + 1
            # 55x55x96 feature map 생성 (55는 (227-11+1)/4)
            # 최종적으로 227 -> 55
            nn.ReLU(), # 96x55x55
            nn.MaxPool2d(kernel_size=3, stride=2)
            # 55 -> (55-3+1)/2 = 26.5 = 27
            # 96x27x27 feature map 생성

        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 1, 2), # in_channels: 96, out_channels: 256, kernel_size=5x5, stride=1, padding=2
            # kernel 수 = 48x5x5 (드롭아웃을 사용했기 때문에 96/2=48) 형태의 256개
            # 256x27x27
            nn.ReLU(),
            nn.MaxPool2d(3, 2) # 27 -> 13
            # 256x13x13
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU() # 13 유지
            # 384x13x13
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU() # 13 유지
            # 384x13x13
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2) # 13 -> 6
            # 256x6x6
        )

        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x): # input size = 3x227x227
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out) # 64x4096x1x1
        out = out.view(out.size(0), -1) # 64x4096

        out = F.relu(self.fc1(out))
        out = F.dropout(out, 0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, 0.5)
        out = self.fc3(out)
        out = F.log_softmax(out, dim=1)

        return out

In [8]:
model = fashion_mnist_alexnet().to(device) # to()로 모델에 gpu 사용
criterion = F.nll_loss # nll_loss : negative log likelihood loss
optimizer = optim.Adam(model.parameters()) # model(신경망) 파라미터를 optimizer에 전달해줄 때 nn.Module의 parameters() 메소드를 사용

In [9]:
from torchsummary import summary as summary_

summary_(model, (1,227,227), batch_size)
# summary_: (model, input_size, batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [512, 96, 55, 55]          11,712
              ReLU-2          [512, 96, 55, 55]               0
         MaxPool2d-3          [512, 96, 27, 27]               0
            Conv2d-4         [512, 256, 27, 27]         614,656
              ReLU-5         [512, 256, 27, 27]               0
         MaxPool2d-6         [512, 256, 13, 13]               0
            Conv2d-7         [512, 384, 13, 13]         885,120
              ReLU-8         [512, 384, 13, 13]               0
            Conv2d-9         [512, 384, 13, 13]       1,327,488
             ReLU-10         [512, 384, 13, 13]               0
           Conv2d-11         [512, 256, 13, 13]         884,992
             ReLU-12         [512, 256, 13, 13]               0
        MaxPool2d-13           [512, 256, 6, 6]               0
           Linear-14                [51

In [10]:
def train(model, device, train_loader, optimizer, epoch):
    model.train() # 모델을 학습 모드로 설정 # 모델 내부에서 드롭아웃 및 배치 정규와 등 활성화 되도록
    for batch_idx, (data, target) in enumerate(train_loader):
        # enumberate() : 인덱스와 원소로 이루어진 튜플(tuple)을 만들어줌
        target = target.type(torch.LongTensor) # 모델의 출력과 타겟 간의 데이터 형식을 일치시키기 위해 필요함
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 항상 backpropagation 하기전에 미분(gradient)을 zero로 만들어주고 시작해야 한다.
        # 각 배치마다 모델의 파라미터 gd를 누적하는 것을 방지
        output = model(data)
        loss = criterion(output, target) # criterion = loss_fn
        loss.backward() # Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # step() : 파라미터를 업데이트함 # 경사 하강법 수행하는 단계
        if (batch_idx + 1) % 30 == 0: # 30번째 미니배치 마다 현재 학습 상황을 출력
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [11]:
def test(model, device, test_loader):
    model.eval() # 모델을 평가 모드로 설정 # 모델 내부에서 드롭아웃 및 배치 정규화 등 비활성화 되도록
    test_loss = 0
    correct = 0 # 정확도 초기화
    with torch.no_grad(): # test 중에는 기울기 계산 안해도 되니까 # 메모리 절약 가능
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item() #reduction='sum' 은 각 미니배치에서 손실을 모두 더한다는 것
            pred = output.max(1, keepdim=True)[1] # 모델의 출력 중 가장 큰 값의 인덱스 사용하여 예측 값 계산
            correct += pred.eq(target.view_as(pred)).sum().item() # 맞은 예측의 개수

        test_loss /= len(test_loader.dataset)  # -> mean
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
        print('='*50)

###pred = output.max(1, keepdim=True)[1]

- output.max(1, keepdim=True)[1]은 각 이미지에 대해 가장 큰 값을 찾고 그 값을 (값, 인덱스) 형태로 반환, keepdim=True는 차원 유지 상태로 값 반환
- [1]는 반환된 튜플의 두 번째 요소, 즉 가장 큰 값의 인덱스 선택
- 따라서 pred는 모델의 예측값 중 가장 큰 값을 갖는 클래스의 인덱스를 담고 있는 리스트!!
- 가장 큰 값 == 확률이 가장 높은 것

---------

### 실제 클래스 레이블(target)과 모델의 예측 결과(pred) 예시
target = [2, 0, 1, 4, 3]  # 각 이미지의 실제 클래스 레이블
pred = [2, 0, 1, 3, 3]    # 모델의 예측 결과

---------

###correct += pred.eq(target.view_as(pred)).sum().item()
- pred.eq(target.view_as(pred))는 pred와 target 비교한 boolean 값 리스트 생성
- sum으로 true 개수 더하고
- item으로 파이썬의 정수로 변호나

---------

따라서 예시 기준으로 코드를 수행하면
pred.eq(target.view_as(pred)) =>  [True, True, True, False, True]

In [13]:
for epoch in range(1, epochs+1):
    train(model, device, training_loader, optimizer, epoch)
    test(model, device, validation_loader)

Train Epoch:1 [1856/60000 (3%)]	Loss: 0.242897
Train Epoch:1 [3776/60000 (6%)]	Loss: 0.156035
Train Epoch:1 [5696/60000 (9%)]	Loss: 0.102052
Train Epoch:1 [7616/60000 (13%)]	Loss: 0.305565
Train Epoch:1 [9536/60000 (16%)]	Loss: 0.173029
Train Epoch:1 [11456/60000 (19%)]	Loss: 0.026302
Train Epoch:1 [13376/60000 (22%)]	Loss: 0.313172
Train Epoch:1 [15296/60000 (25%)]	Loss: 0.127456
Train Epoch:1 [17216/60000 (29%)]	Loss: 0.122151
Train Epoch:1 [19136/60000 (32%)]	Loss: 0.112499
Train Epoch:1 [21056/60000 (35%)]	Loss: 0.151660
Train Epoch:1 [22976/60000 (38%)]	Loss: 0.112900
Train Epoch:1 [24896/60000 (41%)]	Loss: 0.191683
Train Epoch:1 [26816/60000 (45%)]	Loss: 0.175339
Train Epoch:1 [28736/60000 (48%)]	Loss: 0.253973
Train Epoch:1 [30656/60000 (51%)]	Loss: 0.180366
Train Epoch:1 [32576/60000 (54%)]	Loss: 0.096055
Train Epoch:1 [34496/60000 (57%)]	Loss: 0.298041
Train Epoch:1 [36416/60000 (61%)]	Loss: 0.157342
Train Epoch:1 [38336/60000 (64%)]	Loss: 0.115836
Train Epoch:1 [40256/60000 (

코드 참조: https://ingu627.github.io/code/alexnet_pytorch/